In [4]:
# %conda update conda
# %conda update --all
# %conda install pytorch torchvision torchaudio -c pytorch
%pip install torch
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
print(torch.__version__)
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import Word2Vec
import numpy as np
import re

Note: you may need to restart the kernel to use updated packages.


NameError: name 'torch' is not defined

In [ ]:
!conda update -n base -c defaults conda --force -y

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::conda==23.5.2=py311hca03da5_0
  - defaults/osx-arm64::conda-libmamba-solver==23.5.0=py311hca03da5_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.11.0



## Package Plan ##

  environment location: /Users/vrajpatel/miniconda3

  added / updated specs:
    - conda


The following NEW packages will be INSTALLED:

  brotli-python      pkgs/main/osx-arm64::brotli-python-1.0.9-py311h313beb8_8 
  requests           pkgs/main/osx-arm64::requests-2.32.3-py311hca03da5_1 
  setuptools         pkgs/main/osx-arm64::setuptools-75.1.0-py311hca03da5_0 

The following packages will be UPDATED

In [18]:
!conda install pytorch torchvision torchaudio -c pytorch


Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::conda==23.5.2=py311hca03da5_0
  - defaults/osx-arm64::conda-libmamba-solver==23.5.0=py311hca03da5_0
unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-arm64::conda==23.5.2=py311hca03da5_0
  - defaults/osx-arm64::conda-libmamba-solver==23.5.0=py311hca03da5_0
done


==> WARNING: A newer version of conda exists. <==
  current version: 23.5.2
  latest version: 24.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda u

## Extracting Language Features

In [ ]:
df = pd.read_csv('../Data/transcripts.csv', header=None, names=['Participant', 'Transcript'])

# Clean the transcripts by removing "Interviewer" parts
cleaned_transcripts = []
for transcript in df['Transcript']:
    split_sentences = re.split(r'(?<=\|)', transcript)
    cleaned_sentences = [re.sub(r'Interviewer:.*?\|', '', s).strip().replace('|', '') for s in split_sentences if s.strip()]
    cleaned_transcripts.append(' '.join(cleaned_sentences))

df['Transcript'] = cleaned_transcripts

In [ ]:
# Extracting syntactic features using CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english', min_df=2, max_df=0.8)
count_matrix = count_vectorizer.fit_transform(df['Transcript'])
count_df = pd.DataFrame(count_matrix.toarray(), columns=count_vectorizer.get_feature_names_out())

# Plotting the top 10 words by count
top_10_words = count_df.sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_words.values, y=top_10_words.index)
plt.title('Top 10 Words by Count')
plt.xlabel('Count')
plt.ylabel('Words')
plt.show()

In [ ]:
# Extracting syntactic features using TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=2, max_df=0.8)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Transcript'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Plotting the top 10 words by TF-IDF score
top_10_tfidf_words = tfidf_df.sum().sort_values(ascending=False).head(10)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_tfidf_words.values, y=top_10_tfidf_words.index)
plt.title('Top 10 Words by TF-IDF Score')
plt.xlabel('TF-IDF Score')
plt.ylabel('Words')
plt.show()

# Displaying results
print("Count Vectorizer Matrix:")
print(count_df.head())

In [ ]:
# Extracting semantic features using Vader sentiment analysis
analyzer = SentimentIntensityAnalyzer()
df['Sentiment'] = df['Transcript'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Plotting sentiment scores
plt.figure(figsize=(10, 6))
sns.histplot(df['Sentiment'], bins=20, kde=True)
plt.title('Sentiment Scores Distribution')
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.show()

print("\nSentiment Scores:")
print(df[['Participant', 'Sentiment']].head())

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the cleaned transcripts
encoded_inputs = tokenizer(cleaned_transcripts, padding=True, truncation=True, return_tensors='pt')

# Get the BERT embeddings
with torch.no_grad():
    outputs = model(**encoded_inputs)
    bert_embeddings = outputs.last_hidden_state.mean(dim=1).numpy()

bert_df = pd.DataFrame(bert_embeddings)
print("\nBERT Embeddings:")
print(bert_df.head())